In [3]:
# https://github.com/kvsingh/music-mood-classification

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
import joblib

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import spotify_lyrics_scraper as spotify

# Get the SP_DC token
token = spotify.getToken("AQBSz33_ODmldHiskIdlUiuu4yS92-tR4swS5FeqJuHYfBNUdEs3j0mWK4Hmo9wDc3eTK6IRAE0xFoXnH94CkXHKUWvZuFLTdTxyBdOhLCtSyhZLRWM21tgwJDAI1MxFAA_dEPWMtVMxGd9GefWoL4JbtxCXlsfKvk0CF5zpBQ1m_JskXi7hHLj-wLJBlNdTlpdkeA_LZwBCZG5yQowbiE4hO8K5")

def get_lyrics(song_name):
    lyrics_data = spotify.getLyrics(token, songName=song_name)
    if lyrics_data['status']:
        lyrics_lines = lyrics_data['message']['lyrics']['lines']
        lyrics = " ".join([line['words'] for line in lyrics_lines])
        return lyrics
    else:
        return ""

# Add a new column with the lyrics
df['lyrics'] = df['name'].apply(lambda x: get_lyrics(x))

# Load the pre-trained model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define the emotions
emotions = [
    'happy',
    'anger',
    'fear',
    'surprise',
    'sadness',
    'disgust'
]

def get_emotion_probabilities(lyrics):
    # Tokenize the input text
    inputs = tokenizer(lyrics, return_tensors="pt", truncation=True, padding=True)

    # Get the model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract logits and compute probabilities
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)[0]

    # Map emotions to probabilities
    emotion_probs = {emotion: float(probabilities[idx]) for idx, emotion in enumerate(emotions)}
    return emotion_probs

# Load the CSV file
df = pd.read_csv('../songs/data_moods.csv')

# Combine all the lyrics lines into a single string for each song and get emotion probabilities
df['emotion_probabilities'] = df['lyrics'].apply(lambda x: get_emotion_probabilities(x))

# Save the updated DataFrame back to the CSV file
df.to_csv('../songs/data_moods.csv', index=False)

In [59]:
df = pd.read_csv('../songs/data_moods.csv')
df

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.7300,0.000000,0.0843,0.6250,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.8320,0.196000,0.1530,0.1660,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.1390,0.000077,0.0934,0.1160,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.4380,0.000006,0.1130,0.5870,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.9320,0.000000,0.1370,0.4450,-3.604,0.1060,169.881,1,4,Energetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,windcatcher,windcatcher,Leo Nocta,59VApBbrS2IADQk4ml5mdo,2020-06-19,36,123066,0.402,0.96100,0.2360,0.919000,0.0921,0.1460,-20.615,0.0603,129.736,0,3,Calm
682,yellow is the color of her eyes,yellow is the color of her eyes,Soccer Mommy,4D3nttJPU6L0M2epr7sId6,2019-11-19,5,435080,0.452,0.75700,0.5150,0.120000,0.1400,0.1910,-7.351,0.0255,80.537,11,4,Sad
683,you broke me first,you broke me first,Tate McRae,45bE4HXI0AwGZXfZtMp8JR,2020-04-17,87,169265,0.642,0.78600,0.3740,0.000000,0.0906,0.0799,-9.386,0.0545,124.099,4,4,Sad
684,you were good to me,brent,Jeremy Zucker,4CxFN5zON70B3VOPBYbd6P,2019-05-03,76,219146,0.561,0.91300,0.0848,0.000026,0.1120,0.2060,-15.099,0.0404,102.128,2,4,Sad


In [60]:
moods= {'Sad' : 0, 'Calm' : 1, 'Energetic' : 2, 'Happy' : 3}
df['mood'] = df['mood'].apply(lambda x: moods[x])
df['mood'].value_counts()

0    197
1    195
2    154
3    140
Name: mood, dtype: int64

In [61]:
df['release_date'] = pd.to_datetime(df['release_date'])
#df['release_year'] = df['release_date'].dt.year
#df['release_month']= df['release_date'].dt.month
#df['release_day']= df['release_date'].dt.day

X = df.drop(['mood','id','name','album','artist','release_date'],axis=1)
y = df['mood']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X

,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature
0,68,379266,0.866,0.13700,0.7300,0.000000,0.0843,0.6250,-8.201,0.0767,118.523,5,4
1,43,318800,0.381,0.01890,0.8320,0.196000,0.1530,0.1660,-5.069,0.0492,120.255,8,4
2,60,217946,0.346,0.91300,0.1390,0.000077,0.0934,0.1160,-15.326,0.0321,136.168,0,4
3,2,233000,0.466,0.08900,0.4380,0.000006,0.1130,0.5870,-12.858,0.0608,193.100,4,4
4,60,268000,0.419,0.00171,0.9320,0.000000,0.1370,0.4450,-3.604,0.1060,169.881,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,36,123066,0.402,0.96100,0.2360,0.919000,0.0921,0.1460,-20.615,0.0603,129.736,0,3
682,5,435080,0.452,0.75700,0.5150,0.120000,0.1400,0.1910,-7.351,0.0255,80.537,11,4
683,87,169265,0.642,0.78600,0.3740,0.000000,0.0906,0.0799,-9.386,0.0545,124.099,4,4
684,76,219146,0.561,0.91300,0.0848,0.000026,0.1120,0.2060,-15.099,0.0404,102.128,2,4


In [62]:
print(X_train.shape)
print(X_test.shape)

(548, 13)
(138, 13)


In [63]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(X_train)

nn = MLPClassifier(max_iter = 15000, alpha=1.0, hidden_layer_sizes=8)
scores = cross_val_score(nn, train_scaled, y_train, cv=5)
print ("cv score: " + str(scores.mean()))

hyper_opt = False
if hyper_opt:
    params = {"alpha": np.logspace(-4, 2, 7), 'hidden_layer_sizes': [1, 2, 5, 10, 20, 50, 100]}
    clf = GridSearchCV(nn, params)
    clf.fit(train_scaled, y_train)
    print("hyperparam optimized score : " + str(clf.best_score_))

cv score: 0.8156296914095078


In [64]:
clf.best_estimator_

MLPClassifier(alpha=1.0, hidden_layer_sizes=100, max_iter=15000)

In [65]:
clf.best_params_

{'alpha': 1.0, 'hidden_layer_sizes': 100}

In [66]:
results = cross_validate(nn, train_scaled, y_train, cv=10, return_train_score=True, return_estimator=True)
results


{'fit_time': array([0.59050488, 0.61053276, 0.69574094, 0.6633637 , 0.54342914,
        0.70126796, 0.502635  , 0.53977108, 0.46534181, 0.57470298]),
 'score_time': array([0.00032592, 0.00037813, 0.00035191, 0.00032425, 0.00032592,
        0.00038886, 0.00031996, 0.00032687, 0.00030708, 0.00045609]),
 'estimator': [MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000)],
 'test_score': array([0.

In [67]:
nn = MLPClassifier(hidden_layer_sizes=8, max_iter=15000, alpha=1.0)
nn.fit(train_scaled, y_train)
test_preds = nn.predict(scaler.transform(X_test))
accuracy_score(test_preds, y_test)

0.7898550724637681

In [68]:
# safe  model
joblib.dump(nn, "./neural_network.joblib")

['./neural_network.joblib']

In [69]:
nn.predict(scaler.transform(X_test))

array([1, 2, 2, 2, 3, 0, 1, 2, 1, 0, 3, 0, 1, 2, 0, 0, 1, 3, 3, 1, 0, 0,
       3, 2, 3, 0, 0, 1, 0, 2, 0, 1, 2, 3, 3, 3, 1, 1, 0, 1, 0, 2, 1, 0,
       2, 1, 2, 2, 2, 0, 0, 2, 3, 3, 0, 1, 1, 3, 1, 2, 0, 1, 0, 1, 1, 2,
       2, 0, 0, 2, 2, 0, 2, 2, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 1, 2, 0, 3,
       0, 1, 3, 3, 1, 2, 0, 2, 2, 3, 0, 1, 0, 2, 1, 0, 0, 1, 2, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 2, 0, 3, 0, 2, 2, 0, 1, 1, 1, 3, 3, 2, 0, 1,
       3, 1, 1, 0, 1, 2])

## Lets see if we can leverage the accuracy with more data

### Ger more data

In [70]:
keys = df.keys()
keys =keys.to_list()
#keys.remove('mood')

sad_songs = pd.read_csv('../songs/very_sadSongs.csv',index_col=None)
calm_songs = pd.read_csv('../songs/very_calmSongs.csv',index_col=None)
energetic_songs = pd.read_csv('../songs/very_energeticSongs.csv',index_col=None)
happy_songs = pd.read_csv('../songs/very_happySongs.csv',index_col=None)

df_existing = df.copy()

new_songs = [sad_songs, calm_songs, energetic_songs, happy_songs]
moods = [0,1,2,3]

for i in range(len(new_songs)):
    new_df = new_songs[i]
    new_df = new_df.reset_index(drop=True)
    new_df['mood'] = moods[i]
    df_existing = pd.concat([df_existing,new_df],ignore_index=True)
    

df_existing = df_existing.drop(['id','name','album','artist','release_date','Unnamed: 0'],axis=1)



In [71]:
X = df_existing.drop(['mood'],axis=1)
y = df_existing['mood']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [72]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(X_train)

nn = MLPClassifier(max_iter = 15000, alpha=1.0, hidden_layer_sizes=8)
scores = cross_val_score(nn, train_scaled, y_train, cv=5)
print ("cv score: " + str(scores.mean()))

hyper_opt = True
if hyper_opt:
    params = {"alpha": np.logspace(-4, 2, 7), 'hidden_layer_sizes': [1, 2, 5, 10, 20, 50, 100]}
    clf = GridSearchCV(nn, params)
    clf.fit(train_scaled, y_train)
    print("hyperparam optimized score : " + str(clf.best_score_))

cv score: 0.7717300712461352
hyperparam optimized score : 0.8088318322355155


In [73]:
clf.best_estimator_

MLPClassifier(alpha=1.0, hidden_layer_sizes=100, max_iter=15000)

In [74]:
clf.best_params_

{'alpha': 1.0, 'hidden_layer_sizes': 100}

In [75]:
results = cross_validate(nn, train_scaled, y_train, cv=10, return_train_score=True, return_estimator=True)
results

{'fit_time': array([1.01678324, 0.796242  , 0.98170805, 1.02307606, 0.89058304,
        1.00307989, 0.88434219, 0.93762207, 1.02510095, 1.03690481]),
 'score_time': array([0.00037789, 0.00033998, 0.0003109 , 0.00045419, 0.00036883,
        0.00032592, 0.00034094, 0.00030589, 0.00031996, 0.00033903]),
 'estimator': [MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000),
  MLPClassifier(alpha=1.0, hidden_layer_sizes=8, max_iter=15000)],
 'test_score': array([0.

In [89]:
nn = MLPClassifier(hidden_layer_sizes=100, max_iter=15000, alpha=1.0)
nn.fit(train_scaled, y_train)
test_preds = nn.predict(scaler.transform(X_test))
accuracy_score(test_preds, y_test)

0.8564814814814815

In [91]:
nn2 = MLPClassifier(hidden_layer_sizes=100, max_iter=15000, alpha=1.0)
nn2.fit(train_scaled, y_train)
test_preds = nn.predict(scaler.transform(X_test))
accuracy_score(test_preds, y_test)

0.8564814814814815

In [90]:
# safe  model
joblib.dump(nn, "./neural_network_86.joblib")

['./neural_network_86.joblib']